<a href="https://colab.research.google.com/github/animesh182/Final-Year-Project/blob/main/ganfull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTING LIBRARIES

In [146]:
import os
import math
import numpy as np
import torch
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow import keras
import pathlib
from keras.layers import LeakyReLU
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import Reshape
from keras.layers import PReLU
from IPython import display
import random
from matplotlib import pyplot



DEFINING DISCRIMINATOR MODEL

In [125]:
def get_discriminator_model(channels=1):
    
    input = (96,96,channels)
    model = Sequential()
    filter=[64,64,128,128,256,256,512,512]
    for id, size in enumerate(filter):

      model.add(layers.Conv2D(size,kernel_size = 3, strides=id % 2 + 1, padding="same"))
      if(id!=0):
        model.add(BatchNormalization())
      model.add(LeakyReLU(alpha=0.2))


    model.add(Flatten())
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation="sigmoid"))

    return model




                                                                      DEFINING GENERATOR MODEL

In [147]:

def get_generator_model(channels=1):           # Main Method that returns gen model
  input = (24,24,channels)
  model= Sequential()
  model.add(layers.Conv2D(64,kernel_size=9,strides=1,padding="same"))
  model.add(PReLU())
  NewModel= get_generator_residual(model)
  FinalModel = get_generator_shuffler(NewModel)
  return FinalModel

  
def get_generator_residual(model, kernel_size=3, filters=64, stride=1):
  iterator = np.arange(1,6)
  for x in iterator:
    model.add(layers.Conv2D(filters,kernel_size, stride, padding="same"))
    model.add(BatchNormalization())
    if x!=6:
      model.add(PReLU())
      model.add(layers.Conv2D(filters,kernel_size, stride, padding="same"))
      model.add(PReLU())
  
  return model

def get_generator_shuffler(model, kernel_size=3,filters=256,stride=1,upscale_factor=2):
  iterator = np.arange(1,2)
  image_size=24
  for  x in iterator:
    image_size*=upscale_factor
    model.add(layers.Conv2D(filters,kernel_size,stride,padding="same"))
    model.add(Reshape((image_size,image_size )))
    model.add(PReLU())
  return model




DEFINING ACTUAL GAN MODEL

In [127]:
def get_gan_model(gen_model, dis_model):
  dis_model.trainable = False
  model = Sequential()
  model.add(gen_model)
  model.add(dis_model)
  optimz = keras.optimizers.Adam(learning_rate=0.01)
  model.compile(loss="binary_crossentropy",optimizer=optimz,metrics=['accuracy'])
  return model

                        DATA SOURCE FOR MODEL

In [128]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [129]:
root_dir = "/gdrive/MyDrive/dataset"
root_dir = pathlib.Path(root_dir)


                    SPLITTING DATA INTO VALIDATION AND TRAINING

In [130]:
crop_size = (24, 24)
batch_size = 64
upscale_factor = 4
input_size = 24 

train_ds = image_dataset_from_directory(
    root_dir,
    batch_size=batch_size,
    image_size=crop_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
    label_mode=None,
)

valid_ds = image_dataset_from_directory(
    root_dir,
    batch_size=batch_size,
    image_size=crop_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    label_mode=None,
)


Found 5160 files belonging to 1 classes.
Using 4128 files for training.
Found 5160 files belonging to 1 classes.
Using 1032 files for validation.


                              PREPPING DATASET FOR MODEL INPUT AND COMPARISON



In [131]:
def normalize(images):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images

# The map function applies the normalize function to each element in the train
# and test datasets
train_ds =  train_ds.map(normalize)

valid_ds  =  valid_ds.map(normalize)

# The first time you use the dataset, the images will be loaded from disk
# Caching will keep them in memory, making training faster

def process_input(input, input_size, upscale_factor):
    input = tf.image.rgb_to_yuv(input)
    last_dimension_axis = len(input.shape) - 1
    y, u, v = tf.split(input, 3, axis=last_dimension_axis)
    return tf.image.resize(y, [input_size, input_size], method="area")


def process_target(input):
    input = tf.image.rgb_to_yuv(input)
    last_dimension_axis = len(input.shape) - 1
    y, u, v = tf.split(input, 3, axis=last_dimension_axis)
    return y

def get_lowres_image(img, upscale_factor):
    return img.resize((img.size[0] // upscale_factor, img.size[1] // upscale_factor), PIL.Image.BICUBIC,)

train_ds = train_ds.map(
    lambda x: (process_input(x, input_size=24, upscale_factor=3), process_target(x))
)
train_ds = train_ds.prefetch(buffer_size=32)

valid_ds = valid_ds.map(
    lambda x: (process_input(x, input_size=24, upscale_factor=3), process_target(x))
)
valid_ds = valid_ds.prefetch(buffer_size=32)
print(train_ds)#


<PrefetchDataset element_spec=(TensorSpec(shape=(None, 24, 24, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 24, 24, 1), dtype=tf.float32, name=None))>


                                                REAL(SUPER RESOLUTIONED SAMPLES FROM GENERATOR

In [132]:
def real_sample_generator(dataset, samples):
  rd = 0
  selected_real= 0 
  numbering = 0
  for x in dataset:
    y = np.asarray(x[1], dtype='object')
    rd = np.random.randint(0,int(y.shape[0]),int(samples))
    selected_real = y[rd].astype('float32')
    numbering = np.ones((samples,1))
    print("...")
  return selected_real, numbering


        FAKE SAMPLES FOR DISCRIMINATOR AKA REAL SAMPLES FROM GENERATOR




In [133]:
def fake_sample_generator(gen_model, images, samples):
  dataset = get_lowres_image(images,upscale_factor)
  for x in dataset:
     fake_samples= gen_model.predict(x)
     numbering = np.ones((samples,1))
     print("...")
  return fake_samples, numbering
    

TESTING GENERATED SAMPLES

In [134]:
def save_plot(examples, epoch, n=10):
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
	# save plot to file
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()

EVALUATING THE DISCRIMINATOR AND SAVING MODEL




In [135]:
def model_evaluation(gen_model,dis_model, dataset, epoch, iteration = 100, batch =256):
    half_batch = int(batch_size/2)
    for i in range(iteration):
        x_real,y_real = real_sample_generator(dataset, half_batch)
        print("finished")
        _,real_accuracy = dis_model.evaluate(x_real,y_real,verbose=0)
        print("finished accuracy calculation")
        x_fake,y_fake = fake_sample_generator(gen_model,dataset,half_batch)
        print("finished2")
        _,fake_accuracy = dis_model.train_on_batch(x_fake,y_fake)
        print('>%d real= %.0f%% fake = %.0f%%'%(i+1,real_accuracy*100,fake_accuracy*100))
        save_plot(x_fake, epoch)


        filename = 'generator_model_%03d.h5' % (epoch + 1)
        gen_model.save(filename)

TRAINING THE MODEL

In [155]:
def train(gen_model,dis_model, gan_model, dataset,epochs=100,batch=256):
  batch_per_epoch=int(dataset[2].shape[0]/batch)
  half_batch=int(batch/2)
  for x in range(batch_per_epoch):
    X_real, y_real = real_sample_generator(dataset, half_batch)
    X_fake, y_fake = fake_sample_generator(dataset , half_batch)
    X, y = np.vstack((X_real, X_fake)), np.vstack((y_real, y_fake))
    d_loss, _ = dis_model.train_on_batch(X, y)
    X_gan = get_lowres_image(train_ds, batch)
    y_gan = np.ones((batch, 1))
    g_loss = gan_model.train_on_batch(X_gan, y_gan)
    print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, batch_per_epoch, d_loss, g_loss))
    if (i+1) % 10 == 0:
      model_evaluation(gen_model, dis_model, dataset, x)

RUNNING THE MODEL

In [156]:
dis_model=get_discriminator_model()
gen_model=get_generator_model()
gan_model=get_gan_model(gen_model,dis_model)
train(gen_model,dis_model, gan_model, train_ds)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 24, 24, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 24, 24, 1), dtype=tf.float32, name=None))>


TypeError: ignored